# E08_캐글 경진대회 무작정 따라하기

In [ ]:
# $ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/kaggle-kakr-housing-data.zip
$ mv kaggle-kakr-housing-data.zip ~/aiffel/e/e08_kaggle_kakr_housing
$ cd ~/aiffel/e/e08_kaggle_kakr_housing
$ unzip kaggle-kakr-housing-data.zip

# 중간에 데이터 변동이 있어 그냥 제출시 길이 안맞는 에러가 발생한다.

## Baseline 모델 - setting

In [ ]:
# 시각화를 위한 matplotlib import 
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 필요 라이브러리 import
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns

print('얍💢')

In [ ]:
# 데이터 경로 지정
data_dir = os.getenv('HOME')+'/aiffel/e/e08_kaggle_kakr_housing/data'

train_data_path = join(data_dir, 'train.csv')
sub_data_path = join(data_dir, 'test.csv')      # 테스트, 즉 submission 시 사용할 데이터 경로

print(train_data_path)
print(sub_data_path)

## Baseline 모델 - 데이터 이해하기

In [ ]:
# 데이터 불러오기
data = pd.read_csv(train_data_path)
sub = pd.read_csv(sub_data_path)
print('train data dim : {}'.format(data.shape))
print('sub data dim : {}'.format(sub.shape))

# train data dim : (15035, 21)
# sub data dim : (6468, 20)

In [ ]:
# 학습데이터 라벨 제거
y = data['price']
del data['price']

print(data.columns)

# 전체 데이터 탐색을 위한 train, test 데이터 합치기
# model 학습시 다시 분리해야하므로 병합 전 train_len에 training data개수 저장하여 추후 학습데이터만 불러올 수 있는 인덱스로 사용

train_len = len(data)
data = pd.concat((data, sub), axis=0)

print(len(data))

In [ ]:
# 간단한 전처리 - missingno
msno.matrix(data)

In [ ]:
# 결측치의 개수 출력
for c in data.columns:
    print('{} : {}'.format(c, len(data.loc[pd.isnull(data[c]), c].values)))

In [ ]:
# id컬럼 제거 but 예측결과 제출을 대비한 sub_id변수에 id 컬럼 저장하고 지우기
sub_id = data['id'][train_len:]
del data['id']

print(data.columns)

# data column apply()로 필요한 부분만 잘라내기
data['date'] = data['date'].apply(lambda x : str(x[:6]))

data.head()

# str(x[:6]) : 20141013T000000 형식 데이ㅓ에서 연/월 데이터만 사용하기 위해 자르는것

In [ ]:
# 각 변수별 분포 확인

# 전체 데이터 분포 확인 - 컬럼의 분포가 치우쳤다면 다듬기 작업
# id column 제외한 19 컬럼에 대ㅐ해 한번에 모든 그래프 그리기

# sns.kdeplot 사용
# kdeplot : diescrete 데이터 또한 부드러운 곡선으로 전체 분포 확인할 수 있는 시각화 함수
fig, ax = plt.subplots(10, 2, figsize=(12, 60))   # 가로스크롤 때문에 그래프 확인이 불편하다면 figsize의 x값을 조절해 보세요. 

# id 변수는 제외하고 분포를 확인
count = 0
columns = data.columns
for row in range(10):
    for col in range(2):
        sns.kdeplot(data[columns[count]], ax=ax[row][col])
        ax[row][col].set_title(columns[count], fontsize=15)
        count += 1
        if count == 19 :
            break

- bedrooms, sqft_living, sqft_lot, sqft_above, sqft_basement 변수가 한쪽으로 치우친 경향
- 치우친 분포 -> 로그 변환을 통해 데이터 분포를 정규분포에 가깝게 만든다
    
- 치우친 컬럼들을 `skew_columns` 리스트 안에 넣고 모두 `np.log1p()` 를 활용하여 로그 변환을 진행
- `numpy.log1p()` 함수는 입력 배열 각 요소에 대해 자연로그 log(1 + x)을 반환해 주는 함수

In [ ]:
skew_columns = ['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']

for c in skew_columns:
    data[c] = np.log1p(data[c].values)

print('얍💢')

In [ ]:
# 변환 후 분포
fig, ax = plt.subplots(3, 2, figsize=(12, 15))

count = 0
for row in range(3):
    for col in range(2):
        if count == 5:
            break
        sns.kdeplot(data[skew_columns[count]], ax=ax[row][col])
        ax[row][col].set_title(skew_columns[count], fontsize=15)
        count += 1

In [ ]:
# 로그변환이 분포의 치우침을 줄이는 원리는?
# 로그 함수의 형태를 보면 알 수 있다
xx = np.linspace(0, 10, 500)
yy = np.log(xx)

plt.hlines(0, 0, 10)
plt.vlines(0, -5, 5)
plt.plot(xx, yy, c='r')
plt.show()

로그 함수 특징
- $0 < x < 1$ 범위에서는 기울기가 매우 가파릅니다. 즉, $x$의 구간은 $(0, 1)$로 매우 짧은 반면, $y$의 구간은 $(-\infty, 0)$으로 매우 큽니다.
- 따라서 0에 가깝게 모여있는 값들이 $x$로 입력되면, 그 함수값인 $y$ 값들은 매우 큰 범위로 벌어지게 됩니다. 즉, 로그 함수는 0에 가까운 값들이 조밀하게 모여있는 입력값을, 넓은 범위로 펼칠 수 있는 특징을 가집니다.
- 반면, $x$값이 점점 커짐에 따라 로그 함수의 기울기는 급격히 작아집니다. 이는 곧 큰 $x$값들에 대해서는 $y$값이 크게 차이나지 않게 된다는 뜻이고, 따라서 넓은 범위를 가지는 $x$를 비교적 작은 $y$값의 구간 내에 모이게 하는 특징을 가집니다.    
이러한 특성으로 인해 한 쪽에 몰린 분포에 로그 변환을 취하면 넓게 퍼진다

In [ ]:
# `data[price]` 분포를 로그 변환
sns.kdeplot(y)
plt.show()

- 위 분포를 로그변환하게 되면 어떤 모양일까?
- 위 분포는 0 쪽으로 매우 심하게 치우쳐져 있는 분포를 보인다. 즉, 0과 1000000 사이에 대부분의 값들이 몰려있고, 아주 소수의 집들이 굉장히 높은 가격을 보인다.

- 따라서 이 분포에 로그 변환을 취하면, 0에 가깝게 몰려있는 데이터들은 넓게 퍼질 것이고, 매우 크게 퍼져있는 소수의 데이터들은 작은 y값으로 모일 것이다.

- 즉, 왼쪽으로 치우친 값들은 보다 넓은 범위로 고르게 퍼지고 오른쪽으로 얇고 넓게 퍼진 값들은 보다 작은 범위로 모이게 되므로 전체 분포는 정규분포의 형상을 띄는 방향으로 변환될 것이다.

In [ ]:
y_log_transformation = np.log1p(y)

sns.kdeplot(y_log_transformation)
plt.show()

In [ ]:
# 로그 변환이 필요한 데이터에 대한 처리 완료
# 전체데이터를 다시 나눈다

# `train_len`이 인덱스가 되어 :train_len까지는 학습 데이터, train_len: 부터는 테스트 데이터이므로 `sub` 변수에 저장
sub = data.iloc[train_len:, :]
x = data.iloc[:train_len, :]

print(x.shape)
print(sub.shape)

# 8-7. 일단 제출하고 시작해! Baseline 모델 (4) 모델 설계
---
- 모델링 : baseline 커널은 여러 모델을 함께 사용하여 결과를 섞는 블렌딩(blending) 기법을 활용한다
- 블렌딩(or 앙상블 기법) -> 하나의 개별모델이 아닌 다양한 모델들을 종합하여 결과를 얻는 기법
    - 하나의 강한 머신러닝 알고리즘보다 여러 개의 약한 머신러닝 알고리즘이 낫다
    - img, video, voice 등의 비정형 데이터 분류는 딥러닝이 뛰어나나, 대부분의 정형 데이터 분류시에는 앙상블의 성능이 뛰어나다
    - voting, bagging, boosting, stacking 등이 있다
    - [수비니움 블로그 - 앙상블 기법](https://subinium.github.io/introduction-to-ensemble-1/#:~:text=%EC%95%99%EC%83%81%EB%B8%94(Ensemble)%20%ED%95%99%EC%8A%B5%EC%9D%80%20%EC%97%AC%EB%9F%AC,%EB%A5%BC%20%EA%B0%80%EC%A7%80%EA%B3%A0%20%EC%9D%B4%ED%95%B4%ED%95%98%EB%A9%B4%20%EC%A2%8B%EC%8A%B5%EB%8B%88%EB%8B%A4.)

In [ ]:
# 여러 모델들의 결과를 산술평균하여 블렌딩 모델을 만든다
gboost = GradientBoostingRegressor(random_state=2019)
xgboost = xgb.XGBRegressor(random_state=2019)
lightgbm = lgb.LGBMRegressor(random_state=2019)

models = [{'model':gboost, 'name':'GradientBoosting'}, {'model':xgboost, 'name':'XGBoost'},
          {'model':lightgbm, 'name':'LightGBM'}]

print('얍💢')

In [ ]:
# 교차 검증을 통해 모델의 성능을 평가한다
def get_cv_score(models):
    kfold = KFold(n_splits=5, random_state=2019).get_n_splits(x.values)
    for m in models:
        print("Model {} CV score : {:.4f}".format(m['name'], np.mean(cross_val_score(m['model'], x.values, y)), 
                                                  kf=kfold))
print('얍💢')

In [ ]:
get_cv_score(models)

In [ ]:
# submission file 만들기
# cross_val_score() 함수는 회귀모델 전달시 결정계수인 R^2 점수를 반환
# R^2 값이 1에 가까울수록 모델이 잘 학슴됨을 의미한다
# 3개 트리 모델이 모두 훈련 데이터에 대해 괜찮은 성능을 보여준다

# baseline 모델에선 여러모델을 입력하면 각 모델에대한 예측 결과를 평균내어주는 
# AveragingBlending() 함수를 만들어 사용하며, models 딕셔너리 안의 모델을 모두 x, y로 학습시킨 뒤 predictions에 예측결과값을 모아서 평균한 값을 반환한다
def AveragingBlending(models, x, y, sub_x):
    for m in models : 
        m['model'].fit(x.values, y)
    
    predictions = np.column_stack([
        m['model'].predict(sub_x.values) for m in models
    ])
    return np.mean(predictions, axis=1)

print('얍💢')

In [ ]:
# 예측값 생성
y_pred = AveragingBlending(models, x, y, sub)
print(len(y_pred))
y_pred

In [ ]:
# 제출할 csv파일의 샘플인 data/sample_submission.csv 확인
data_dir = os.getenv('HOME')+'/aiffel/e/e08_kaggle_kakr_housing/data'

submission_path = join(data_dir, 'sample_submission.csv')
submission = pd.read_csv(submission_path)
submission.head()

In [ ]:
# id, price 두 가지 열로 구성되어있으므로 동일한 데이터프레임을 만들어준다
result = pd.DataFrame({
    'id' : sub_id, 
    'price' : y_pred
})

result.head()

In [ ]:
# 제출파일 만들기
my_submission_path = join(data_dir, 'submission.csv')
result.to_csv(my_submission_path, index=False)

print(my_submission_path)

# 8-8. 일단 제출하고 시작해! Baseline 모델 (5) 캐글에 첫 결과 제출하기
---
- 이미 종료된 대회이므로 `Late Submission`만 가능하다    
```kaggle competitions submit -c 2019-2nd-ml-month-with-kakr -f submission.csv -m "Message"```
- 제출하면 score가 뜬다
- score : 120031.23722점
- `Jump to your position on the leaderboard` 클릭시 내 등수로 이동한다

# 8-9. 랭킹을 올리고 싶다면? (1) 다시 한 번, 내 입맛대로 데이터 준비하기
---
## 최적의 모델을 찾아서, hyperparameter 튜닝
---
- 랭킹을 올리기 위해 직접 다양한 Hyperparameter를 튜닝해보면서 모델 성능 개선


### 파라미터(model parameter) vs 하이퍼파라미터?
---
모델 파라미터는 모델이 학습을 하면서 점차 최적화되는, 그리고 최적화가 되어야 하는 파라미터입니다.

예를 들어 선형 회귀의 경우 y_pred = W*x + b 라는 식으로 예측값을 만들어 낼 텐데, 여기에서 모델 파라미터는 W 입니다. 모델은 학습 과정을 거치면서 최적의 y_pred 값, 즉 y_true에 가장 가까운 값을 출력해낼 수 있는 최적의 W를 찾아나갈 것입니다.

반면, 하이퍼 파라미터는 모델이 학습을 하기 위해서 사전에 사람이 직접 입력해주는 파라미터입니다.

이는 모델이 학습하는 과정에서 변하지 않습니다. 예를 들어 학습 횟수에 해당하는 epoch 수, 가중치를 업데이트 할 학습률(learning rate), 또는 선형 규제를 담당하는 labmda 값 등이 이에 해당합니다.

### 다시 한 번 내 입맛대로 데이터 준비하기
---

In [ ]:
data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'

train_data_path = join(data_dir, 'train.csv')
test_data_path = join(data_dir, 'test.csv') 

train = pd.read_csv(train_data_path)
test = pd.read_csv(test_data_path)

print('얍💢')

In [ ]:
# data lookup
train.head()

In [ ]:
# date 전처리해주기 (to int)
# 모델이 date 또한 예측을 위한 특성으로 활용할 수 있다
train['date'] = train['date'].apply(lambda i: i[:6]).astype(int)
train.head()

In [ ]:
# 타겟 데이터에 해당하는 price 컬럼 저일하기
# y 변수에 price를 넣어두고 train에선 삭제하기
y = train['price']
del train['price']

print(train.columns)

# id컬럼도 삭제해두기
del train['id']

print(train.columns)

In [ ]:
# 위 작업을 test데이터에 대해서도 동일하게!
# 단, price가 없으므로 price처리는 해주지 않아도 된다
test['date'] = test['date'].apply(lambda i: i[:6]).astype(int)

del test['id']

print(test.columns)

In [ ]:
# target data check
y

In [ ]:
# 가격 데이터 분포 확인하기
sns.kdeplot(y)
plt.show()
# ==> 왼쪽으로 크게 치우쳐 있는 형태를 보인다

In [ ]:
# thus, y는 np.log1p()함수를 통해 로그 변환을 해주고 추후 모델이 값을 예측하고 나면 다시 np.exp1m()을 활용해서 되돌린다
# np.exp1m()은 np.log1p()와는 반대로 각 원소 x마다 exp(x)-1의 값을 반환해준다
y = np.log1p(y)
y

In [ ]:
sns.kdeplot(y)
plt.show()
# ==> 비교적 완만한 정규분포의 형태로 잘 변환되었다

In [ ]:
# info()함수로 전체 데이터의 자료형을 한 눈에 확인한다
train.info()
# ==> 모두 실수||정수 자료형으로 문제 없이 모델 학습에 활용할 수 있다

## 8-10. 랭킹을 올리고 싶다면? (2) 다양한 실험을 위해 함수로 만들어 쓰자
- 본격적으로 모델 튜닝 들어가기
- ML모델을 학습시키고 튜닝하다보면 실험해볼 것들이 워낙 많아서 시간이 호다닥 간다
- 보다 다양하고 많은 실험 <-- 실험을 위한 도구들이 준비되어있어야 유리
- thus,반복작업들은 함수로 먼저 만들어두고 맘껏 실험하자

In [ ]:
# RMSE 계산
# 필요 라이브러리 import
# train/test/valid dataset으로 나누기 위한 train_test_split, RMSE점수 계산을 위한 mean_squared_error import

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

print('얍💢')

In [ ]:
# 대회 점수 평가 척도인 RMSE 계산을 위한 함수 제작
# warning) y_test, y_pred는 위에서 np.log1p()로 변환이 된 값이므로 원 데이터 단위에 맞게 되돌리기 위해 np.expm1()을 추가해야함
# exp로 다시 변환하여 mse를 계산한 값에 np.sqrt를 취하면 RMSE값을 얻을 수 있다

def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred)))

print('얍💢')

In [ ]:
# XGBRegressor, LGBMRegressor, GradientBoostingRegressor, RandomForestRegressor 모델 가져오기

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

print('얍💢')

In [ ]:
# 모델 인스턴스 생성 후 models라는 리스트에 입력
# model param초기화나 dataset 구성에 사용되는 랜덤 시드값인 random_state값을 fixed로 하거나, 지정하지않고 None으로 세팅할 수 있다
# random_state값이 fixed인 경우 모델==데이터셋 동일한 경우 ML학습결과도 항상 동일하게 재현된다
# None으로 남겨두면 모델 내부에서 랜덤 시드값을 임의선택하므로 결과적으로 param초기화나 데이터셋 구성 양상이 달라져서
# 모델과 데이터셋이 동일하더라도 ML 학습결과는 학습할 때마다 달라진다

# baseline부터 시작해서 여러 실험을 통해 성능개선을 검증하자
# a trial이 성능향상에 유효한지 여부 판단을 위해선 랜덤적 요소/변화에서 생기는 불확실성을 제거해야하므로
# random_state값을 fixed한다.
# fix 안했을 때 어떻게 될 지 궁금하면 random_state값을 None으로 남겨두고 실험을 반복하면 된다

In [ ]:
# random_state는 모델초기화나 데이터셋 구성에 사용되는 랜덤 시드값입니다. 
#random_state=None    # 이게 초기값입니다. 아무것도 지정하지 않고 None을 넘겨주면 모델 내부에서 임의로 선택합니다.  
random_state=2020        # 하지만 우리는 이렇게 고정값을 세팅해 두겠습니다. 

gboost = GradientBoostingRegressor(random_state=random_state)
xgboost = XGBRegressor(random_state=random_state)
lightgbm = LGBMRegressor(random_state=random_state)
rdforest = RandomForestRegressor(random_state=random_state)

models = [gboost, xgboost, lightgbm, rdforest]

print('얍💢')

In [ ]:
# 각 모델의 이름 얻는 법) 클래스의 __name__속성에 접근
gboost.__class__.__name__

In [ ]:
# 이름 접근가능하다면 for문안에서 각 모델별로 학습 및 예측이 가능하다
df = {}

for model in models:
    # 모델 이름 획득
    model_name = model.__class__.__name__

    # train, test 데이터셋 분리 - 여기에도 random_state를 고정합니다. 
    X_train, X_test, y_train, y_test = train_test_split(train, y, random_state=random_state, test_size=0.2)

    # 모델 학습
    model.fit(X_train, y_train)
    
    # 예측
    y_pred = model.predict(X_test)

    # 예측 결과의 rmse값 저장
    df[model_name] = rmse(y_test, y_pred)
    
    # data frame에 저장
    score_df = pd.DataFrame(df, index=['RMSE']).T.sort_values('RMSE', ascending=False)
    
df

In [ ]:
# 간단히 네 가지 모델에 대해 모두 RMSE값을 빠르게 얻을 수 있음
# get_scores(models, train, y)함수로 만들어보기
def get_scores(models, train, y):
    df = {}
    for m in models:
      m_name = m.__class__.__name__
      
      X_train, X_test, y_train, y_test = train_test_split(train, y, random_state=random_state, test_size=0.2)
      m.fit(X_train, y_train)
      y_pred = m.predict(X_test)
      df[m_name] = rmse(y_test, y_pred)
      
      score_df = pd.DataFrame(df, index=['RMSE']).T.sort_values('RMSE', ascending=True)
      
    return score_df
get_scores(models, train, y)

# 8-11. 랭킹을 올리고 싶다면? (3) 하이퍼 파라미터 튜닝의 최강자, 그리드 탐색
---
- 이제 모델과 데이터셋이 있다면 RMSE결과값을 나타내주는 함수가 준비되었으므로, 다양한 하이퍼파라미터로 실험해보자
- sklearn.model_selection라이브러리 안에 있는 GridSearchCV 클래스 활용
- `GridSearchCV` 란? [Random Search vs Grid Search](https://shwksl101.github.io/ml/dl/2019/01/30/Hyper_parameter_optimization.html)

In [ ]:
# 실험을 위해 
from sklearn.model_selection import GridSearchCV

print('얍💢')